In [4]:
#!/usr/bin/env python
# encoding:utf-8
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
import subprocess
import sklearn
from sklearn.externals import joblib
from sklearn.feature_selection import  f_classif


def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0)
seq=[]

RFH_feature=pd.read_csv('/home02/chenhuangrong/RFH_feature.csv',header=None,index_col=False)
RFH_feature=RFH_feature.values
RFH_positive_feature=RFH_feature[:len(RFH_feature)/2]
RFH_negative_feature=RFH_feature[len(RFH_feature)/2:]

RNA_code='ACGU'
k=1
interval=1
select_num=40
divided_num=10.0
division_num=10

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
kf = KFold(n_splits=division_num,shuffle=False)  
for select_num in xrange(30,204):
    y_pred_prob_all=[]
    y_pred_all=[]
    Y_all=[]
    ACC_all=0
    precision_all=0
    recall_all=0
    SN_all=0
    SP_all=0
    GM_all=0
    TP_all=0
    TN_all=0
    FP_all=0
    FN_all=0
    F_measure_all=0
    F1_Score_all=0
    pos_all=0
    neg_all=0
    MCC_all=0
    num_mark=0
    for train_index , test_index in kf.split(RFH_positive_feature):  
        num_mark=num_mark+1 #
        RFH_postive_train_x=pd.DataFrame(RFH_positive_feature).iloc[train_index,:]
        RFH_postive_test_y=pd.DataFrame(RFH_positive_feature).iloc[test_index,:]
        RFH_negative_train_x=pd.DataFrame(RFH_negative_feature).iloc[train_index,:]
        RFH_negative_test_y=pd.DataFrame(RFH_negative_feature).iloc[test_index,:]
        RFH_train_x=np.concatenate([RFH_postive_train_x,RFH_negative_train_x])
        RFH_test_y=np.concatenate([RFH_postive_test_y,RFH_negative_test_y])

        
        scaler=sklearn.preprocessing.MinMaxScaler()
        X_train_=scaler.fit_transform(RFH_train_x)        
        X_train=np.array(X_train_)
        Y_train = list(map(lambda x: 1, xrange(len(X_train) / 2)))
        Y2_train = list(map(lambda x: 0, xrange(len(X_train) / 2)))
        Y_train.extend(Y2_train)
        Y_train = np.array(Y_train)
        F, pval = f_classif(X_train, Y_train)
        idx = np.argsort(F)
        selected_list_=idx[::-1]
        selected_list_=selected_list_[xrange(select_num)]
        X_train=pd.DataFrame(X_train)
        X_train=X_train.iloc[:,selected_list_]
        X_train=X_train.values
       
        X_test_=scaler.transform(RFH_test_y)
        X_test=pd.DataFrame(X_test_)
        X_test=X_test.iloc[:,selected_list_]
        X_test=X_test.values
        Y_test  = list(map(lambda x: 1, xrange(len(X_test) / 2)))
        Y2_test  = list(map(lambda x: 0, xrange(len(X_test) / 2)))
        Y_test.extend(Y2_test )
        Y_test  = np.array(Y_test)


        print "data_prepared"
        print "X_train.shape",X_train.shape
        print "X_test.shape",X_test.shape
        
        svc = svm.SVC(probability=True)
        parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
        clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
        clf.fit(X_train, Y_train)
#         joblib.dump(clf, '/home02/chenhuangrong/another_F_score_RFH_PSNP%d.model_%d'%(num_mark,select_num))
        C=clf.best_params_['C']
        y_pred_prob=clf.predict_proba(X_test)

        gamma=clf.best_params_['gamma']
        print 'c:',C,'gamma:',gamma
        y_pred=clf.predict(X_test)

        y_pred_prob_all.extend(y_pred_prob)
        y_pred_all.extend(y_pred)
        Y_all.extend(Y_test)
        ACC=metrics.accuracy_score(Y_test,y_pred)
        print ACC
        precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
        F1_Score=metrics.f1_score(Y_test, y_pred)
        F_measure=F1_Score
        MCC=metrics.matthews_corrcoef(Y_test, y_pred)

        pos=TP+FN
        neg=FP+TN
        ACC_all=ACC_all+ACC
#         print "ACC_all:",ACC_all
        precision_all=precision_all+precision
        recall_all=recall_all+recall
        SN_all=SN_all+SN
        SP_all=SP_all+SP
        GM_all=GM_all+GM
        TP_all=TP_all+TP
        TN_all=TN_all+TN
        FP_all=FP_all+FP
        FN_all=FN_all+FN
        F_measure_all=F_measure_all+F_measure
        F1_Score_all=F1_Score_all+F1_Score
        pos_all=pos_all+pos
        neg_all=neg_all+neg
        MCC_all=MCC_all+MCC
    all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
#     pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/another_F_score_PsCPm6APred_independent_test_with_F_Score%d_%d.csv'%(num_mark,select_num),header=None,index=False)
    fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
    roc_auc = auc(fpr, tpr)  
    savedata=[str(X_train.shape[1])+"select_num:"+str(select_num),"正"+str(pos_all)+'负'+str(neg_all),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
                SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
                FN_all,FP_all,TN_all]
    print savedata
    final_out_to_excel.append(savedata)
pd.DataFrame(final_out_to_excel).to_excel("/home02/chenhuangrong/RFH_F_score_%d.xlsx"%select_num,sheet_name="F_score",index=False,header=False)

data_prepared
X_train.shape (2352, 30)
X_test.shape (262, 30)
c: 2.82842712475 gamma: 0.03125
0.751908396947
data_prepared
X_train.shape (2352, 30)
X_test.shape (262, 30)
c: 1.25992104989 gamma: 0.0701538780193
0.782442748092
data_prepared
X_train.shape (2352, 30)
X_test.shape (262, 30)
c: 0.561231024155 gamma: 0.157490131237
0.767175572519
data_prepared
X_train.shape (2352, 30)
X_test.shape (262, 30)
c: 6.34960420787 gamma: 0.0701538780193
0.748091603053
data_prepared
X_train.shape (2352, 30)
X_test.shape (262, 30)
c: 2.82842712475 gamma: 0.0701538780193
0.687022900763
data_prepared
X_train.shape (2352, 30)
X_test.shape (262, 30)
c: 0.561231024155 gamma: 0.157490131237
0.729007633588
data_prepared
X_train.shape (2352, 30)
X_test.shape (262, 30)
c: 2.82842712475 gamma: 0.03125
0.763358778626
data_prepared
X_train.shape (2354, 30)
X_test.shape (260, 30)
c: 6.34960420787 gamma: 0.03125
0.723076923077
data_prepared
X_train.shape (2354, 30)
X_test.shape (260, 30)
c: 1.25992104989 gamma: 0.

KeyboardInterrupt: 